# Making $$$ with ML:

You've got $1000 to burn. You've decided you want to invest in the stock market, specifically Tesla.
Let's see if we can use Machine Learning to optimize our returns.

Download TSLA.csv from here: https://www.kaggle.com/timoboz/tesla-stock-data-from-2010-to-2020/data

Let's get started.

Let's first do some Exploratory Data Analysis (EDA) on the file we've got.

This file is a comma-separated values (CSV) file with 7 columns.

The columns are:
* Date
* Opening price
* Highest price that day
* Lowest price that day
* Closing price
* Adjusted closing price, taking splits etc into account
* Trading volume



In [0]:
# Importing pandas. "pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language."
import pandas as pd 			  		 			 	 	 		 		 	  		   	  			  	
pd.options.display.max_rows = 999
# Read in the CSV, save it to a pandas dataframe variable called 'tsla_data'.
tsla_data = pd.read_csv("TSLA.csv");

In [0]:
# .head() gives us the first 5 rows of the data frame.
# You can also pass .head() a parameter to return any number of rows. Like .head(10) for 10 rows.
tsla_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900


In [0]:
# .shape tells us the number of rows, and the number of columns.
# This dataset has 2416 rows, and 7 columns.
# The NYSE and NASDAQ average about 253 trading days a year. 
# This is from 365.25 (days on average per year) * 5/7 (proportion work days per week) - 6 (weekday holidays) - 3*5/7 (fixed date holidays) = 252.75 ≈ 253.
# 10 * 253 = 2530, this dataset is pretty close. Let's assume it's not missing any days.
tsla_data.shape

(2416, 7)

This is 10 years of data, with information about the stock starting from 2010. 

Let's make some assumptions for the sake of time, we're not hedge fund managers yet.

**Assumptions**

*   We can only place one order a day (buy or sell), for the entire amount held.
*   If we place an order, we assume it will go through at that price.
*   We start with $1000

We're going to track a few key pieces of information.

*   Money in wallet
*   Number of stocks held

Let's start with just 2010, to see how much money we would have made if we started with $1000 on the first day of this file. 



In [0]:
# We're going to just pull the 2010 data. I like sticking this in variable, and array, because we'll likely do this again, and by multiple years.
years_to_pull = [2010]

# Let's tell pandas to treat the 'Date' column as a date.
tsla_data['Date'] = pd.to_datetime(tsla_data['Date'])

# Let's make a function for re-use
def pull_data_by_year(tsla_data, years_to_pull):
  tsla_data_by_year = tsla_data[tsla_data['Date'].dt.year.isin(years_to_pull)]
  return tsla_data_by_year

tsla_data_by_year = pull_data_by_year(tsla_data, years_to_pull)
tsla_data_by_year.shape

(130, 7)

In [0]:
# Sort by date ASC
tsla_data_by_year = tsla_data_by_year.sort_values(by = 'Date')

Let's add a couple columns to help us with the data. I want to see tomorrow's adjusted close, and I want to know if it's higher than today's adjusted close.

In [0]:
tsla_data_by_year["Adj Close Tomorrow"] = tsla_data_by_year["Adj Close"].shift(-1)
tsla_data_by_year["Stock Goes Up Tomorrow"] = tsla_data_by_year["Adj Close"] < tsla_data_by_year["Adj Close Tomorrow"]
tsla_data_by_year.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Adj Close Tomorrow,Stock Goes Up Tomorrow
0,2010-06-29,19.000000,25.000000,17.540001,23.889999,23.889999,18766300,23.830000,False
1,2010-06-30,25.790001,30.420000,23.299999,23.830000,23.830000,17187100,21.959999,False
2,2010-07-01,25.000000,25.920000,20.270000,21.959999,21.959999,8218800,19.200001,False
3,2010-07-02,23.000000,23.100000,18.709999,19.200001,19.200001,5139800,16.110001,False
4,2010-07-06,20.000000,20.000000,15.830000,16.110001,16.110001,6866900,15.800000,False
5,2010-07-07,16.400000,16.629999,14.980000,15.800000,15.800000,6921700,17.459999,True
6,2010-07-08,16.139999,17.520000,15.570000,17.459999,17.459999,7711400,17.400000,False
7,2010-07-09,17.580000,17.900000,16.549999,17.400000,17.400000,4050600,17.049999,False
8,2010-07-12,17.950001,18.070000,17.000000,17.049999,17.049999,2202500,18.139999,True
9,2010-07-13,17.389999,18.639999,16.900000,18.139999,18.139999,2680100,19.840000,True


Following the rule buy low sell high, and we're looking at historical data, we can say the following.

To start pick the first day whose following day's Adj Close price goes up, and buy $1000 worth of shares on that day.

We'll have 3 positions.

Buy
Sell
Hold

In code:

haveNoStock && !goesUpTomorrow = hold

haveNoStock && goesUpTomorrow = buy

haveStock && !goesUpTomorrow = sell

haveStock && goesUpTomorrow = hold


In [0]:
# Setting some default values
tsla_data_by_year['Position'] = 'Hold'
tsla_data_by_year['Number Of Stocks Held'] = 0
tsla_data_by_year['Money In Wallet'] = 0

In [0]:
tsla_data_by_year.at[0, 'Money In Wallet'] = 1000
tsla_data_by_year.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Adj Close Tomorrow,Stock Goes Up Tomorrow,Position,Number Of Stocks Held,Money In Wallet
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300,23.830000,False,Hold,0,1000
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100,21.959999,False,Hold,0,0
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800,19.200001,False,Hold,0,0
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800,16.110001,False,Hold,0,0
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900,15.800000,False,Hold,0,0


In [0]:
previousRow = ''
for index, row in tsla_data_by_year.iterrows():
  if(index > 0):
    row['Money In Wallet'] = previousRow['Money In Wallet']
    row['Number Of Stocks Held'] = previousRow['Number Of Stocks Held']
  if(row['Number Of Stocks Held'] == 0 and not row['Stock Goes Up Tomorrow']):
    row['Position'] = 'Hold'
    # print(1)
  elif(row['Number Of Stocks Held'] == 0 and row['Stock Goes Up Tomorrow']):
    row['Position'] = 'Buy'
    row['Number Of Stocks Held'] = row['Money In Wallet'] / row['Adj Close']
    row['Money In Wallet'] -= row['Number Of Stocks Held'] * row['Adj Close']
    # print(2)
  elif(row['Number Of Stocks Held'] > 0 and not row['Stock Goes Up Tomorrow']):
    row['Position'] = 'Sell'
    row['Money In Wallet'] += row['Number Of Stocks Held'] * row['Adj Close']
    row['Number Of Stocks Held'] = 0
    # print(3)
  elif(row['Number Of Stocks Held'] > 0 and row['Stock Goes Up Tomorrow']):
    row['Position'] = 'Hold'
    # print(4)
  previousRow = row
  tsla_data_by_year.at[index] = row

In [0]:
tsla_data_by_year = tsla_data_by_year.round(2)
tsla_data_by_year.head(999)

,Date,Open,High,Low,Close,Adj Close,Volume,Adj Close Tomorrow,Stock Goes Up Tomorrow,Position,Number Of Stocks Held,Money In Wallet
0,2010-06-29,19.00,25.00,17.54,23.89,23.89,18766300,23.83,False,Hold,0.00,1000.00
1,2010-06-30,25.79,30.42,23.30,23.83,23.83,17187100,21.96,False,Hold,0.00,1000.00
2,2010-07-01,25.00,25.92,20.27,21.96,21.96,8218800,19.20,False,Hold,0.00,1000.00
3,2010-07-02,23.00,23.10,18.71,19.20,19.20,5139800,16.11,False,Hold,0.00,1000.00
4,2010-07-06,20.00,20.00,15.83,16.11,16.11,6866900,15.80,False,Hold,0.00,1000.00
5,2010-07-07,16.40,16.63,14.98,15.80,15.80,6921700,17.46,True,Buy,63.29,0.00
6,2010-07-08,16.14,17.52,15.57,17.46,17.46,7711400,17.40,False,Sell,0.00,1105.06
7,2010-07-09,17.58,17.90,16.55,17.40,17.40,4050600,17.05,False,Hold,0.00,1105.06
8,2010-07-12,17.95,18.07,17.00,17.05,17.05,2202500,18.14,True,Buy,64.81,0.00
9,2010-07-13,17.39,18.64,16.90,18.14,18.14,2680100,19.84,True,Hold,64.81,0.00


As the end of 2010 we would have $8,645 if we knew the future.


In [0]:
# TODO Next
# Add indicators
# Add Decision Tree